In [1]:
import numpy as np
import pandas as pd
import torch
from FairRanking.datasets.law import Law
from FairRanking.datasets.adult import Adult
from FairRanking.helpers import nDCG_cls, disparate_impact, calc_accuracy, rND_torch, auc_estimator, group_pairwise_accuracy, calc_rnd, disparity_loss

In [44]:
data = Law('Race','/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Compas()
#data = Wiki()
#data.load_data()
(X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data(test_size=0.0001, val_size=0.0001)

In [45]:
s = torch.argmax(torch.tensor(s_train, dtype=torch.float32), dim=1)
y = torch.tensor(y_train.flatten(), dtype=torch.float32)
rND_torch(y, s, step=10, start=10, protected_group_idx=0, non_protected_group_idx=1)

{0: 1282, 1: 18281}


0.379294878214798

In [43]:
y

tensor([0., 0., 0.,  ..., 0., 1., 0.])

In [3]:
df = pd.read_csv('/Users/robert/Desktop/Bachelor/FairRanker/data/law_data.csv')
df.head()

,id,race,gender,LSAT,UGPA,region_first,ZFYA,sander_index,first_pf
0,19954,White,1,48.0,4.2,FW,1.73,1.000000,1
1,20757,White,2,45.0,4.2,GL,-1.07,0.962500,1
2,16564,White,1,41.0,4.2,MW,0.85,0.912500,1
3,5069,White,1,48.0,4.1,NE,-0.96,0.990476,1
4,18884,White,2,48.0,4.1,NE,1.31,0.990476,1


In [33]:
def rND(prediction, s, step=10, start=10, protected_group_idx=1, non_protected_group_idx=0):
    '''
    Computes the normalized Discounted Difference, which is a measure of how different are
    the proportion of members in the protected group at the top-i cutoff and in the overall
    population. Lower is better. 0 is the best possible value. Only binary protected groups
    are supported.
    '''
    s = np.asarray(s)
    if len(s.shape) > 1:
        s = s[:, 0]
    # we don't want to have uniqual size
    if len(prediction) != len(s):
        raise AssertionError(
            'len of prediction ' + str(len(prediction)) + ' and s ' + str(len(s)) + ' are uniqual'
        )
    unique, counts = np.unique(s, return_counts=True)
    count_dict_all = dict(zip(unique, counts))
    try:
        len(unique) == 2
    except AssertionError:
        print('array s contains more than 2 classes.')

    keys = [protected_group_idx, non_protected_group_idx]
    for key in keys:
        if key not in count_dict_all:
            count_dict_all[key] = 0

    sorted_idx = np.argsort(np.array(prediction))[::-1]
    sorted_s = np.array(s[sorted_idx])

    # a fake sorted list of s which gives the worst possible result, used for regularization purposes
    # it is maximally discriminative, having all non-protected individuals first and then the others.
    fake_horrible_s = np.hstack(([non_protected_group_idx for i in range(count_dict_all[non_protected_group_idx])],
                                 [protected_group_idx for i in range(count_dict_all[protected_group_idx])]))

    fake_horrible_s_2 = np.hstack(([protected_group_idx for i in range(count_dict_all[protected_group_idx])],
                                   [non_protected_group_idx for i in range(count_dict_all[non_protected_group_idx])]))

    rnd = 0
    max_rnd = 0
    max_rnd_2 = 0

    for i in range(start, len(s), step):
        unique, counts = np.unique(sorted_s[:i], return_counts=True)
        count_dict_top_i = dict(zip(unique, counts))

        unique, counts = np.unique(fake_horrible_s[:i], return_counts=True)
        count_dict_reg = dict(zip(unique, counts))

        unique_2, counts_2 = np.unique(fake_horrible_s_2[:i], return_counts=True)
        count_dict_reg_2 = dict(zip(unique_2, counts_2))

        keys = [protected_group_idx, non_protected_group_idx]
        for key in keys:
            if key not in count_dict_reg:
                count_dict_reg[key] = 0
            if key not in count_dict_top_i:
                count_dict_top_i[key] = 0
            if key not in count_dict_reg_2:
                count_dict_reg_2[key] = 0
        rnd += 1 / np.log2(i) * np.abs(
            count_dict_top_i[protected_group_idx] / i - count_dict_all[protected_group_idx] / len(s))
        max_rnd += 1 / np.log2(i) * np.abs(
            count_dict_reg[protected_group_idx] / i - count_dict_all[protected_group_idx] / len(s))
        max_rnd_2 += 1 / np.log2(i) * np.abs(
            count_dict_reg_2[protected_group_idx] / i - count_dict_all[protected_group_idx] / len(s))

    max_rnd = max(max_rnd, max_rnd_2)

    return rnd / max_rnd

In [46]:
rND(y_train.flatten(), s_train, protected_group_idx=1, non_protected_group_idx=0)

0.3672382044738496